<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/taxi_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 8.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=4292ef98160fab928ed6bf79ce24d71cdc96521d72c32ef796432c7c3f2bae91
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 27 19:42 kaggle.json


In [3]:
! kaggle competitions download -c taxi

  0% 0.00/60.8k [00:00<?, ?B/s]
100% 60.8k/60.8k [00:00<00:00, 46.9MB/s]


In [4]:
!unzip taxi.zip

Archive:  taxi.zip
  inflating: submission_form.csv     
  inflating: test.csv                
  inflating: train.csv               


In [142]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [143]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
      torch.cuda.manual_seed_all(777)

In [144]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [145]:
cab_type = {"Uber":0, "Lyft":1}

train['cab_type']= train['cab_type'].map(cab_type)
test['cab_type']= test['cab_type'].map(cab_type)

In [146]:
summary_mapping = {" Clear ":0," Drizzle ":1," Foggy ":2," Light Rain ":3," Mostly Cloudy ":4," Overcast ":5," Partly Cloudy ":6," Possible Drizzle ":7," Rain ":8}

train['short_summary'] = train['short_summary'].map(summary_mapping)
test['short_summary'] = test['short_summary'].map(summary_mapping)

In [147]:
icon_mapping = {" clear-day ":0," clear-night ":1," cloudy ":2," fog ":3," partly-cloudy-day ":4," partly-cloudy-night ":5," rain ":6}

train['icon'] = train['icon'].map(icon_mapping)
test['icon'] = test['icon'].map(icon_mapping)

In [148]:
x_data = train.loc[0:,"cab_type":"precipIntensityMax"]
y_data = train.loc[0:,"price"]

x_data = np.array(x_data)
y_data = np.array(y_data)
scaler = MinMaxScaler()

x_data = scaler.fit_transform(x_data)
x_train = torch.FloatTensor(x_data).to(device)
y_train = torch.FloatTensor(y_data).to(device).reshape(-1,1)

print(x_train[:5])
print(x_train.shape)
print(y_train[:5])
print(y_train.shape)

tensor([[0.0000, 0.3059, 0.0000, 0.7150, 0.7110, 0.3750, 0.1990, 0.9400, 0.9138,
         0.2811, 0.1391, 0.2777, 1.0000, 0.8546, 0.0042, 1.0000, 0.0000, 0.4073,
         0.9733],
        [0.0000, 0.3059, 0.0000, 0.9321, 0.9423, 0.6250, 0.0000, 0.0000, 0.2241,
         0.7333, 0.4845, 1.0000, 0.3333, 0.7053, 0.2332, 0.9800, 0.5000, 0.3626,
         0.0034],
        [0.0000, 0.3869, 0.0000, 0.3383, 0.3482, 0.5000, 0.0000, 0.0000, 0.7586,
         0.2502, 0.1221, 0.9663, 0.8333, 0.4883, 0.6702, 0.5200, 0.0000, 0.1233,
         0.0000],
        [0.0000, 0.3869, 0.0000, 0.4962, 0.4387, 0.5000, 0.0000, 0.0000, 0.5690,
         0.5381, 0.4382, 0.9980, 0.8333, 0.5408, 0.1955, 0.7500, 0.0000, 0.7260,
         0.0000],
        [0.0000, 0.3869, 0.0000, 0.6990, 0.7443, 0.2500, 0.0000, 0.0000, 0.9828,
         0.0412, 0.0068, 0.0000, 0.5000, 0.8637, 0.2676, 1.0000, 0.0000, 0.2283,
         0.0027]], device='cuda:0')
torch.Size([2809, 19])
tensor([[32.5000],
        [11.0000],
        [27.0000],
  

In [149]:
x_test = test.loc[0:,"cab_type":]
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
x_test = torch.FloatTensor(x_test).to(device)

x_test[:3]

tensor([[1.0000, 0.0196, 0.0000, 0.6116, 0.5542, 0.5000, 0.0000, 0.0000, 0.5172,
         0.5643, 0.3146, 1.0000, 0.8333, 0.6117, 0.7141, 0.7200, 0.0000, 0.3142,
         0.8746],
        [1.0000, 0.0196, 0.0000, 0.6440, 0.5593, 1.0000, 0.8977, 1.0000, 0.9655,
         0.7924, 0.4212, 0.4383, 1.0000, 0.8090, 0.3346, 1.0000, 0.0000, 0.1982,
         0.8910],
        [1.0000, 0.0196, 0.0000, 0.5069, 0.4613, 0.0000, 0.0000, 0.0000, 0.6379,
         0.4729, 0.2449, 1.0000, 0.1667, 0.5771, 0.0883, 0.0300, 0.0000, 0.4228,
         0.7293]], device='cuda:0')

In [225]:
learning_rate = 1e-4
training_epochs = 1000
batch_size = 100

In [226]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [227]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [228]:
linear1 = torch.nn.Linear(19, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 256, bias=True)
linear4 = torch.nn.Linear(256, 512, bias=True)
linear5 = torch.nn.Linear(512, 1024, bias=True)
linear6 = torch.nn.Linear(1024, 1024, bias=True)
linear7 = torch.nn.Linear(1024, 512, bias=True)
linear8 = torch.nn.Linear(512, 512, bias=True)
linear9 = torch.nn.Linear(512, 256, bias=True)
linear10 = torch.nn.Linear(256, 256, bias=True)
linear11 = torch.nn.Linear(256, 128, bias=True)
linear12 = torch.nn.Linear(128, 128, bias=True)
linear13 = torch.nn.Linear(128, 1, bias=True)
dropout=torch.nn.Dropout(p=0.4)
mish= Mish()


In [229]:
torch.nn.init.kaiming_normal_(linear1.weight)
torch.nn.init.kaiming_normal_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.kaiming_normal_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.kaiming_normal_(linear6.weight)
torch.nn.init.kaiming_normal_(linear7.weight)
torch.nn.init.kaiming_normal_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)
torch.nn.init.kaiming_normal_(linear10.weight)
torch.nn.init.xavier_uniform_(linear11.weight)
torch.nn.init.kaiming_normal_(linear12.weight)
torch.nn.init.kaiming_normal_(linear13.weight)


Parameter containing:
tensor([[ 0.0045,  0.1810,  0.0268, -0.0378,  0.0770,  0.0471,  0.1040,  0.0800,
         -0.2117,  0.0954, -0.1214, -0.0439, -0.0461, -0.0468, -0.2862,  0.1652,
         -0.0480, -0.0602, -0.1216,  0.2009,  0.1046, -0.0256,  0.1502,  0.1451,
          0.1230, -0.0198,  0.0177, -0.0731, -0.0564,  0.0462,  0.0531, -0.0522,
          0.0185,  0.1019,  0.0822,  0.1456, -0.1195, -0.2015,  0.0491, -0.0621,
          0.0666, -0.0289, -0.2149,  0.0757, -0.0337, -0.0558,  0.1716, -0.1890,
         -0.2752, -0.1679,  0.0774,  0.1613,  0.1139, -0.0601,  0.0256, -0.1112,
          0.1489, -0.1403, -0.0771, -0.0233,  0.0298,  0.1769,  0.0322,  0.1476,
         -0.0142, -0.2309,  0.0298,  0.0532, -0.0825,  0.2086, -0.1335,  0.0838,
          0.1716, -0.1121, -0.0728,  0.0070, -0.1245, -0.1031, -0.1335, -0.0967,
         -0.1015, -0.0047,  0.0952, -0.0870, -0.0472,  0.0655,  0.0310,  0.0071,
          0.0064, -0.0121, -0.2491, -0.0702,  0.0664, -0.0485,  0.0254,  0.0053,
      

In [230]:
model = torch.nn.Sequential(linear1,mish,dropout,
                            linear2,mish,dropout,
                            linear3,mish,dropout,
                            linear4,mish,dropout,
                            linear5,mish,dropout,
                            linear6,mish,dropout,
                            linear7,mish,dropout,
                            linear8,mish,dropout,
                            linear9,mish,dropout,
                            linear10,mish,dropout,
                            linear11,mish,dropout,
                            linear12,mish,dropout,
                            linear13).to(device)

In [231]:
loss = torch.nn.MSELoss(reduction='mean').to(device)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [232]:
total_batch = len(data_loader)

for epoch in range(training_epochs+1):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch
        
    if(epoch%10 == 0):    
          print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))
          print(cost.item())

print('Learning finished')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


Epoch: 0000 cost = 634.602416992
527.2874145507812
Epoch: 0010 cost = 256.945953369
173.85531616210938
Epoch: 0020 cost = 188.772033691
209.8612060546875
Epoch: 0030 cost = 167.282546997
226.6951446533203
Epoch: 0040 cost = 151.523391724
382.2876892089844
Epoch: 0050 cost = 149.158798218
104.90754699707031
Epoch: 0060 cost = 132.258026123
126.8762435913086
Epoch: 0070 cost = 138.326354980
104.36235809326172
Epoch: 0080 cost = 134.755096436
123.34765625
Epoch: 0090 cost = 126.233718872
114.6981430053711
Epoch: 0100 cost = 113.856758118
110.94937896728516
Epoch: 0110 cost = 115.705123901
79.51490783691406
Epoch: 0120 cost = 112.129386902
138.16331481933594
Epoch: 0130 cost = 113.219642639
153.98716735839844
Epoch: 0140 cost = 109.537109375
89.39552307128906
Epoch: 0150 cost = 111.438812256
123.0202865600586
Epoch: 0160 cost = 105.386772156
104.71050262451172
Epoch: 0170 cost = 104.771797180
110.18384552001953
Epoch: 0180 cost = 105.254020691
114.22679138183594
Epoch: 0190 cost = 107.6882

In [233]:
x_test = test.loc[0:,"cab_type":]
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
x_test = torch.FloatTensor(x_test).to(device)

x_test[:3]


tensor([[1.0000, 0.0196, 0.0000, 0.6116, 0.5542, 0.5000, 0.0000, 0.0000, 0.5172,
         0.5643, 0.3146, 1.0000, 0.8333, 0.6117, 0.7141, 0.7200, 0.0000, 0.3142,
         0.8746],
        [1.0000, 0.0196, 0.0000, 0.6440, 0.5593, 1.0000, 0.8977, 1.0000, 0.9655,
         0.7924, 0.4212, 0.4383, 1.0000, 0.8090, 0.3346, 1.0000, 0.0000, 0.1982,
         0.8910],
        [1.0000, 0.0196, 0.0000, 0.5069, 0.4613, 0.0000, 0.0000, 0.0000, 0.6379,
         0.4729, 0.2449, 1.0000, 0.1667, 0.5771, 0.0883, 0.0300, 0.0000, 0.4228,
         0.7293]], device='cuda:0')

In [234]:
model(x_train)

tensor([[19.3808],
        [16.3652],
        [19.5360],
        ...,
        [89.8597],
        [65.2854],
        [65.4504]], device='cuda:0', grad_fn=<AddmmBackward>)

In [235]:
with torch.no_grad():
    model.eval()
    x_test=np.array(x_test.cpu())
    x_test=torch.from_numpy(x_test).float().to(device)
    predict=model(x_test)

In [236]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)
result = pd.read_csv('submission_form.csv')

In [237]:
for i in range(len(predict)):
      result['price'][i]=predict[i].item()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [238]:
result['price'] = result['price'].astype(int)

In [239]:
result.to_csv('submit.csv', index=False)

In [240]:
!kaggle competitions submit -c taxi -f submit.csv -m "14010974 이기택"

100% 601/601 [00:05<00:00, 119B/s]
Successfully submitted to SejongAI.텀프로젝트.택시비예측